# Imports

## Imports and Setup

In [18]:
import pathlib
import sys
import os

# In Jupyter notebooks, __file__ is not defined
# Instead, use the current working directory to modify the path
notebook_dir = os.getcwd()
project_root = os.path.abspath(os.path.join(notebook_dir, '..'))
sys.path.insert(0, project_root)
print(f"Added {project_root} to Python path")

import torch
import numpy as np
import pyterrier as pt
from pathlib import Path

# Initialize PyTerrier
if not pt.started():
    pt.init()

from pyterrier.datasets import Dataset
from pyterrier.measures import *
from fast_forward.encoder import TASBEncoder, ContrieverEncoder
from fast_forward.index import OnDiskIndex, Mode
from fast_forward.util import Indexer
from fast_forward.util.pyterrier import FFInterpolate, FFScore
from pyterrier.terrier import Retriever

from fusions.FFTM2C2 import FFTM2C2
from fusions.experiment import fuse_convex_norm

device="cuda:0" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")
print(f"PyTorch version: {torch.__version__}")


Added /home/weicheng/ir-project to Python path
Using device: cuda:0
PyTorch version: 2.6.0+cu124


/tmp/ipykernel_2965/3639574956.py:18: DeprecationWarning: Call to deprecated function (or staticmethod) started. (use pt.java.started() instead) -- Deprecated since version 0.11.0.
  if not pt.started():


# Dataset Selection

In [19]:
# Dataset Selection: https://pyterrier.readthedocs.io/en/latest/datasets.html
dataset_name = "irds:beir/fiqa"
dataset = pt.get_dataset(dataset_name)
testset = pt.get_dataset(dataset_name + "/test")

# Indexing
indexer = pt.IterDictIndexer(
    str(Path.cwd()),  # this will be ignored
    type=pt.index.IndexingType.MEMORY,
)
index_ref = indexer.index(dataset.get_corpus_iter(), fields=["text"])

beir/fiqa documents: 100%|██████████| 57638/57638 [00:07<00:00, 7256.75it/s]


# Model Configuration

Setting up three retrieval models:
1. BM25 - Classic lexical retrieval
2. TASB - Neural retriever model
3. Contriever - Neural retriever model

In [20]:
# BM25
bm25 = pt.terrier.Retriever(index_ref, wmodel="BM25")
tasb_q_encoder =  tasb_d_encoder = TASBEncoder(device=device)
con_q_encoder = con_d_encoder = ContrieverEncoder(device=device)

In [21]:
safe_dataset_name = dataset_name.replace(":", "_").replace("/", "_")
# Define index paths for both models
tasb_index_path = Path(f"../indexes/ffindex_{safe_dataset_name}_tasb.h5")
con_index_path = Path(f"../indexes/ffindex_{safe_dataset_name}_con.h5")

def load_or_create_index(index_path: pathlib.Path, q_encoder, d_encoder):
    print(index_path.exists())
    try:
        ff_index = OnDiskIndex.load(
            index_path,
            query_encoder=q_encoder,
            mode=Mode.MAXP,
        )
    except FileNotFoundError:
        index_path.parent.mkdir(exist_ok=True, parents=True)
        ff_index = OnDiskIndex(
            index_path,
            query_encoder=q_encoder,
            mode=Mode.MAXP,
        )
        from fast_forward.util import Indexer

        def docs_iter():
            for d in dataset.get_corpus_iter():
                yield {"doc_id": d["docno"], "text": d["text"]}

        Indexer(ff_index, d_encoder, batch_size=8).from_dicts(docs_iter())

    return ff_index.to_memory()

tasb_index = load_or_create_index(tasb_index_path, tasb_q_encoder, tasb_d_encoder)
con_index = load_or_create_index(con_index_path, con_q_encoder, con_d_encoder)



True


100%|██████████| 57638/57638 [00:00<00:00, 2437574.17it/s]


True


100%|██████████| 57638/57638 [00:00<00:00, 2669397.27it/s]


## Create Retrieval Pipelines

We create three pipelines:
1. BM25 only
2. BM25 re-ranked with TASB
3. BM25 re-ranked with Contriever

In [22]:
ff_tasb = FFScore(tasb_index)
ff_con = FFScore(con_index)
RANK_CUTOFF = 50  # Number of documents to retrieve with BM25 before re-ranking

# Define retrieval pipelines
pipeline_0 = (bm25 % RANK_CUTOFF)  # BM25 only
pipeline_1 = bm25 % RANK_CUTOFF >> ff_tasb  # BM25 + TASB re-ranking
pipeline_2 = bm25 % RANK_CUTOFF >> ff_con  # BM25 + Contriever re-ranking


In [23]:

def get_pipeline_result(pipeline: Retriever, ds: Dataset):
    return pipeline.transform(ds.get_topics())

res_1 = get_pipeline_result(pipeline_0,testset)
res_2 = get_pipeline_result(pipeline_1,testset)
res_3 = get_pipeline_result(pipeline_2,testset)


14:00:24.975 [main] WARN org.terrier.querying.ApplyTermPipeline -- The index has no termpipelines configuration, and no control configuration is found. Defaulting to global termpipelines configuration of 'Stopwords,PorterStemmer'. Set a termpipelines control to remove this warning.


## Model Fusion

Combine results from the three models using convex normalization,
with weights 0.2 for BM25, 0.4 for TASB, and 0.4 for Contriever.

In [24]:
fuse_min_max = fuse_convex_norm(
    df1=res_1,
    df2=res_2,
    df3=res_3,
    w1=0.2,
    w2=0.4,
    w3=0.4,
    normalization_method_1="min_max",
    normalization_method_2="min_max",
    normalization_method_3="min_max",
)

fuse_t_min_max = fuse_convex_norm(
    df1=res_1,
    df2=res_2,
    df3=res_3,
    w1=0.2,
    w2=0.4,
    w3=0.4,
    normalization_method_1="theoretical_min_max",
    normalization_method_2="theoretical_min_max",
    normalization_method_3="theoretical_min_max",
)

fuse_z_score = fuse_convex_norm(
    df1=res_1,
    df2=res_2,
    df3=res_3,
    w1=0.2,
    w2=0.4,
    w3=0.4,
    normalization_method_1="z_score",
    normalization_method_2="z_score",
    normalization_method_3="z_score",
)

fuse_min_max_lexical = fuse_convex_norm(
    df1=res_1,
    df2=res_2,
    df3=res_3,
    w1=0.2,
    w2=0.4,
    w3=0.4,
    normalization_method_1="min_max",
    normalization_method_2="unnormalized",
    normalization_method_3="unnormalized",
)

fuse_t_min_max_lexical = fuse_convex_norm(
    df1=res_1,
    df2=res_2,
    df3=res_3,
    w1=0.2,
    w2=0.4,
    w3=0.4,
    normalization_method_1="theoretical_min_max",
    normalization_method_2="unnormalized",
    normalization_method_3="unnormalized",
)

fuse_z_score_lexical = fuse_convex_norm(
    df1=res_1,
    df2=res_2,
    df3=res_3,
    w1=0.2,
    w2=0.4,
    w3=0.4,
    normalization_method_1="z_score",
    normalization_method_2="unnormalized",
    normalization_method_3="unnormalized",
)

fuse_clean = fuse_convex_norm(
    df1=res_1,
    df2=res_2,
    df3=res_3,
    w1=0.2,
    w2=0.4,
    w3=0.4,
    normalization_method_1="unnormalized",
    normalization_method_2="unnormalized",
    normalization_method_3="unnormalized",
)


/home/weicheng/ir-project/env/lib/python3.12/site-packages/pyterrier/model.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(columns=["rank"], errors="ignore", inplace=True)
/home/weicheng/ir-project/env/lib/python3.12/site-packages/pyterrier/model.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["rank"] = df.groupby("qid", sort=False)["score"].rank(ascending=False, method="first").astype(int) -1 + FIRST_RANK
/home/weicheng/ir-project/env/lib/python3.12/site-packages/pyterrier/model.py:31: SettingWithCopyWarning: 
A value is trying to be set on 

## Evaluation Results

Compare performance of individual models vs fusion approach

In [ ]:
pt.Experiment(

    [res_1,res_2,res_3,
    fuse_min_max, 
    fuse_t_min_max, 
    fuse_z_score, 
    fuse_min_max_lexical,
    fuse_t_min_max_lexical, 
    fuse_z_score_lexical,
    fuse_clean],

    testset.get_topics(),
    testset.get_qrels(),
    eval_metrics=[RR @ 100, nDCG @ 100, MAP @ 100],
    names=['bm25', 'bm25+TASB', 'bm25+Cont', 
           'min_max_fusion', 
           'theoretical_min_max_fusion', 
           'z_score_fusion', 
           'min_max_fusion_lexical_only', 
           'theoretical_min_max_fusion_lexical_only', 
           'z_score_fusion_lexical_only',
           'no_normalization_fusion'],
    baseline=0,
    correction="bonferroni",
    save_dir="./results",
)



,name,nDCG@10,AP@100,RR@10,nDCG@10 +,nDCG@10 -,nDCG@10 p-value,nDCG@10 reject,nDCG@10 p-value corrected,AP@100 +,AP@100 -,AP@100 p-value,AP@100 reject,AP@100 p-value corrected,RR@10 +,RR@10 -,RR@10 p-value,RR@10 reject,RR@10 p-value corrected
0,bm25,0.252589,0.206640,0.310271,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN,False,NaN
1,bm25+TASB,0.306769,0.250428,0.371344,204.0,111.0,6.801317e-09,True,6.121185e-08,231.0,142.0,5.456308e-07,True,4.910677e-06,170.0,96.0,3.302430e-06,True,2.972187e-05
2,bm25+Cont,0.278087,0.223183,0.342730,189.0,132.0,8.707841e-03,False,7.837057e-02,228.0,156.0,6.522488e-02,False,5.870239e-01,161.0,113.0,1.663045e-02,False,1.496741e-01
3,min_max_fusion,0.319821,0.261455,0.393266,209.0,81.0,1.019503e-15,True,9.175524e-15,255.0,101.0,2.384721e-12,True,2.146249e-11,173.0,66.0,6.024211e-12,True,5.421789e-11
4,theoretical_min_max_fusion,0.314260,0.254076,0.384178,213.0,69.0,3.508218e-16,True,3.157396e-15,256.0,94.0,5.905687e-12,True,5.315118e-11,174.0,54.0,9.746911e-12,True,8.772220e-11
5,z_score_fusion,0.320009,0.260463,0.393372,214.0,78.0,2.289333e-16,True,2.060400e-15,257.0,97.0,1.758275e-12,True,1.582447e-11,175.0,65.0,2.292300e-12,True,2.063070e-11
6,min_max_fusion_lexical_only,0.309522,0.252317,0.374074,207.0,110.0,8.627872e-10,True,7.765085e-09,232.0,141.0,1.211447e-07,True,1.090302e-06,173.0,94.0,9.530671e-07,True,8.577604e-06
7,theoretical_min_max_fusion_lexical_only,0.309565,0.252361,0.374111,207.0,110.0,8.406669e-10,True,7.566002e-09,232.0,141.0,1.181188e-07,True,1.063069e-06,173.0,94.0,9.394469e-07,True,8.455023e-06
8,z_score_fusion_lexical_only,0.312643,0.256742,0.381692,208.0,93.0,9.540188e-12,True,8.586169e-11,239.0,123.0,9.073385e-10,True,8.166046e-09,172.0,79.0,8.304862e-09,True,7.474375e-08
9,no_normalization_fusion,0.312667,0.254689,0.378361,205.0,68.0,2.765341e-17,True,2.488807e-16,254.0,95.0,2.145978e-12,True,1.931380e-11,163.0,55.0,1.872283e-12,True,1.685055e-11


Use unnormalized fusion of 3 models as baseline here:

In [26]:
result = pt.Experiment(

    [fuse_clean,
    fuse_min_max, 
    fuse_t_min_max, 
    fuse_z_score, 
    fuse_min_max_lexical,
    fuse_t_min_max_lexical, 
    fuse_z_score_lexical,
    pipeline_0,
    pipeline_1,
    pipeline_2,
    ],

    testset.get_topics(),
    testset.get_qrels(),
    eval_metrics=[R@10, R@100, RR @ 100, nDCG @ 100, MAP @ 100],
    names=['no_normalization_fusion',
           'min_max_fusion', 
           'theoretical_min_max_fusion', 
           'z_score_fusion', 
           'min_max_fusion_lexical_only', 
           'theoretical_min_max_fusion_lexical_only', 
           'z_score_fusion_lexical_only',
           'BM25',
            'BM25+TASB',
            'BM25+Cont'
           ],
    baseline=0,
    correction="bonferroni",
)

result.to_csv(f'results/{safe_dataset_name}_output.csv', index=False)

result


,name,R@10,R@100,nDCG@100,AP@100,RR@100,R@10 +,R@10 -,R@10 p-value,R@10 reject,...,AP@100 +,AP@100 -,AP@100 p-value,AP@100 reject,AP@100 p-value corrected,RR@100 +,RR@100 -,RR@100 p-value,RR@100 reject,RR@100 p-value corrected
0,no_normalization_fusion,0.378606,0.485482,0.343885,0.254689,0.384840,NaN,NaN,NaN,False,...,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN,False,NaN
1,min_max_fusion,0.379276,0.485482,0.350686,0.261455,0.399405,39.0,35.0,9.262124e-01,False,...,180.0,129.0,1.249552e-01,False,1.000000e+00,131.0,98.0,5.171275e-02,False,4.654147e-01
2,theoretical_min_max_fusion,0.379612,0.485482,0.344824,0.254076,0.389996,42.0,37.0,8.924883e-01,False,...,168.0,146.0,8.944222e-01,False,1.000000e+00,122.0,110.0,4.987473e-01,False,1.000000e+00
3,z_score_fusion,0.381836,0.485482,0.350018,0.260463,0.399106,42.0,35.0,6.580028e-01,False,...,179.0,129.0,1.780375e-01,False,1.000000e+00,130.0,96.0,5.766141e-02,False,5.189527e-01
4,min_max_fusion_lexical_only,0.376435,0.485482,0.341369,0.252317,0.379960,29.0,37.0,7.377426e-01,False,...,138.0,157.0,6.019936e-01,False,1.000000e+00,111.0,122.0,5.019264e-01,False,1.000000e+00
5,theoretical_min_max_fusion_lexical_only,0.376435,0.485482,0.341404,0.252361,0.379996,29.0,37.0,7.377426e-01,False,...,139.0,157.0,6.088809e-01,False,1.000000e+00,112.0,122.0,5.051579e-01,False,1.000000e+00
6,z_score_fusion_lexical_only,0.374506,0.485482,0.345305,0.256742,0.388083,22.0,32.0,4.793261e-01,False,...,138.0,142.0,5.881552e-01,False,1.000000e+00,107.0,106.0,6.029410e-01,False,1.000000e+00
7,BM25,0.309708,0.485482,0.301535,0.206640,0.320309,24.0,109.0,1.065962e-11,True,...,95.0,254.0,2.145978e-12,True,1.931380e-11,78.0,203.0,6.786087e-12,True,6.107478e-11
8,BM25+TASB,0.372474,0.485482,0.339768,0.250428,0.377691,25.0,39.0,3.311288e-01,False,...,135.0,162.0,3.737192e-01,False,1.000000e+00,107.0,125.0,3.411067e-01,False,1.000000e+00
9,BM25+Cont,0.344599,0.485482,0.318383,0.223183,0.350281,50.0,91.0,1.071167e-03,True,...,147.0,221.0,1.685135e-04,True,1.516622e-03,121.0,185.0,4.332666e-03,True,3.899400e-02
